In [1]:
from transformers import pipeline
import torch
import jsonlines

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [2]:
# classifier = pipeline('text-classification', 'lvwerra/distilbert-imdb', device=device)
classifier = pipeline("text-classification", model="Aron/distilbert-base-uncased-finetuned-emotion", device=device)

Downloading: 100%|██████████| 883/883 [00:00<00:00, 884kB/s]
d:\Anaconda3\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\heegyukim\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Downloading: 100%|██████████| 268M/268M [00:24<00:00, 11.1MB/s] 
Downloading: 100%|██████████| 3

In [8]:
from tqdm.auto import tqdm

# filename = 'gpt-sentiment'
# filename = 'gedi-sentiment+detox'
filename = 'gedi-small-bbc-400-omega60'
items = []

with jsonlines.open(f"GeDi/data/{filename}.jsonl") as f:
  for item in tqdm(f):
    item['prediction'] = classifier(item["generation"])
    items.append(item)

1200it [00:25, 46.41it/s]


In [9]:
import jsonlines
from collections import defaultdict

# ["sadness", "joy", "love", "anger", "fear", "surprise"]
label2id = {
  "LABEL_0": "sadness",
  "LABEL_1": "joy",
  "LABEL_2": "love",
  "LABEL_3": "anger",
  "LABEL_4": "fear",
  "LABEL_5": "surprise"
}

total = defaultdict(lambda: 0)
correct = defaultdict(lambda: 0)

for i, item in tqdm(enumerate(items)):
  label = item['text']
  preds = item['prediction']

  total[label] += len(preds)

  for p in preds:
    if label2id[p['label']] == label.strip():
      correct[label] += 1

for k in sorted(total.keys()):
    print(k, correct[k] / total[k])

1200it [00:00, 463288.37it/s]

anger 0.389
fear 0.517
joy 0.765
love 0.216
sadness 0.679
surprise 0.273


In [7]:
items[0]

{'text': 'sadness',
 'generation': ['about it although no friends we know in company for that matter directoke lives here already and has been around since i immigrated here from kaligad',
  'during our short outing the weather was nice the owners were easy working a response line and feel no need to upset about their habits by removing trail baskets or remote',
  'during bathing i know at times its tempting to feel isolated apart from family and the environment it brings fringe physical and mental health challenges that need to be resolved',
  'over the last months who feels really disheartenedised about key events in development s quiet progress story drama romance soccer covering politics and football with i feel like',
  'over the crash in tsk after years of steady climb but overall gorgeous weather heiplanes again on last flight taking him south an area with lovely trees and'],
 'prediction': [{'label': 'LABEL_0', 'score': 0.902459979057312}]}